In [1]:
library('MVN')
library('caret')
library('readxl')
library('datasets')

Loading required package: ggplot2

Loading required package: lattice



# 6.7

In [2]:
n1 <- 45
print(paste("n1 :", n1))

xbar1 <- matrix(c(204.4, 556.6), nrow = 2, ncol = 1, byrow = TRUE)
print("xbar1 :")
xbar1

S1 <- matrix(c(13825.3, 23823.4, 23823.4, 73107.4), nrow = 2, ncol = 2, byrow = TRUE)
print("S1 :")
S1

[1] "n1 : 45"
[1] "xbar1 :"


204.4
556.6


[1] "S1 :"


13825.3,23823.4
23823.4,73107.4


In [3]:
n2 <- 55
print(paste("n2 :", n2))

xbar2 <- matrix(c(130, 355), nrow = 2, ncol = 1, byrow = TRUE)
print("xbar2 :")
xbar2

S2 <- matrix(c(8632, 19616.7, 19616.7, 55964.5), nrow = 2, ncol = 2, byrow = TRUE)
print("S2 :")
S2

[1] "n2 : 55"
[1] "xbar2 :"


130
355


[1] "S2 :"


8632.0,19616.7
19616.7,55964.5


## (1)	Test the hypothesis H0 : mu1 - mu2 = 0 at the 0.05 significance level

In [4]:
mu <- c(0,0)
T.test <- function(xb1, xb2, S.1, S.2, n1, n2, mu){
  p <- nrow(xb1)
  df1 = p
  df2 = n1 + n2 - p - 1
  alpha <- 0.05
  
  S_pooled <- ((n1 - 1)/(n1 + n2 -2) * S.1) + ((n2 - 1)/(n1 + n2 -2) * S.2)
  
  T_square <- t((xb1 - xb2) - mu) %*% solve(((1/n1) + (1/n2)) * S_pooled) %*% ((xb1 - xb2) - mu)
    
  Fstat <- T_square / (p %*% (n1 + n2 - 2)/(n1 + n2 - p - 1))
  
  Ftable <- qf(1 - alpha, df1 = p, df2 = n1 + n2 - p - 1) %*% (n1 + n2 - 2) %*% p / (n1 + n2 - p - 1)
  
  p_value <- 1 - pf(Fstat, df1 = p, df2 = n1 + n2 - p - 1)
  
  data.frame(T_square = as.numeric(T_square), 
             Fstat = as.numeric(Fstat),
             df1 = p, 
             df2 = n1 + n2 - p - 1, 
             Ftable = as.numeric(Ftable),
             pvalue = as.numeric(p_value),
             row.names="")
}

In [5]:
T.test(xbar1, xbar2, S1, S2, n1, n2, mu) 

,T_square,Fstat,df1,df2,Ftable,pvalue
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
,16.06622,7.951139,2,97,6.244089,0.0006343777


## (2)	Determining the linear combination of mean components for the rejection of H0

In [6]:
S_pooled <- ((n1 - 1)/(n1 + n2 -2) * S1) + ((n2 - 1)/(n1 + n2 -2) * S2)

linear_comb <- solve(S_pooled) %*% (xbar1 - xbar2) 
linear_comb

0.00170252
0.00259163


# 6.8

## Treatment

In [7]:
x1t1 <- c(6, 5, 8, 4, 7)
x2t1 <- c(7, 9, 6, 9, 9)
treatment1 <- rbind(x1t1,x2t1)
treatment1

x1t1,6,5,8,4,7
x2t1,7,9,6,9,9


In [8]:
x1t2 <- c(3, 1, 2)
x2t2 <- c(3, 6, 3)
treatment2 <- rbind(x1t2,x2t2)
treatment2

x1t2,3,1,2
x2t2,3,6,3


In [9]:
x1t3 <- c(2, 5, 3, 2)
x2t3 <- c(3, 1, 1, 3)
treatment3 <- rbind(x1t3,x2t3)
treatment3

x1t3,2,5,3,2
x2t3,3,1,1,3


## a). Break up the observations into mean, treatment, and residual  components, as in (6-35). Construct the corresponding arrays for each variable. (See Example 6.8.)

### X1

In [10]:
#Observations
obs_x1 <- c(x1t1, x1t2, x1t3)
print(paste("Observation vector : "))
obs_x1

sumsquare_obsx1 <- sum(obs_x1^2)
print(paste("Sum-square Observation vector : ",sumsquare_obsx1))

[1] "Observation vector : "


[1] 6 5 8 4 7 3 1 2 2 5 3 2

[1] "Sum-square Observation vector :  246"


In [11]:
#Means
meanx1 <- mean(obs_x1)
print(paste("Mean Vector : "))
(mean_x1 <- c(rep(meanx1, length(x1t1)), 
              rep(meanx1, length(x1t2)), 
              rep(meanx1, length(x1t3))))

sumsquare_meanx1 <- sum(mean_x1^2)
print(paste("Sum-Square Mean Vector : ", sumsquare_meanx1))

[1] "Mean Vector : "


[1] 4 4 4 4 4 4 4 4 4 4 4 4

[1] "Sum-Square Mean Vector :  192"


In [12]:
#Treatment Effects
x1t1_bar <- mean(x1t1)
x1t2_bar <- mean(x1t2)
x1t3_bar <- mean(x1t3)
treateffect_x1t1 <- x1t1_bar - meanx1
treateffect_x1t2 <- x1t2_bar - meanx1
treateffect_x1t3 <- x1t3_bar - meanx1
print(paste("Treatment Effect Vector : "))
(treateffect_x1 <- c(rep(treateffect_x1t1, length(x1t1)), 
                     rep(treateffect_x1t2, length(x1t2)), 
                     rep(treateffect_x1t3, length(x1t3))))

sumsquare_treateffectx1 <- sum(treateffect_x1^2)
print(paste("Sum-square Treatment Effect Vector : ", sumsquare_treateffectx1))

[1] "Treatment Effect Vector : "


[1]  2  2  2  2  2 -2 -2 -2 -1 -1 -1 -1

[1] "Sum-square Treatment Effect Vector :  36"


In [13]:
#Residuals
x1bar <- c(rep(x1t1_bar, length(x1t1)), 
                rep(x1t2_bar, length(x1t2)), 
                rep(x1t3_bar, length(x1t3)))

residual_x1 <- obs_x1 - x1bar
print(paste("Treatment Effect Vector : "))
residual_x1

sumsquare_residx1 <- sum(residual_x1^2)
print(paste("Sum-square Treatment Effect Vector : ", sumsquare_residx1))

[1] "Treatment Effect Vector : "


[1]  0 -1  2 -2  1  1 -1  0 -1  2  0 -1

[1] "Sum-square Treatment Effect Vector :  18"


### X2

In [14]:
#Observations
obs_x2 <- c(x2t1, x2t2, x2t3)
print(paste("Observation vector : "))
obs_x2

sumsquare_obsx2 <- sum(obs_x2^2)
print(paste("Sum-square Observation vector : ", sumsquare_obsx2))

[1] "Observation vector : "


[1] 7 9 6 9 9 3 6 3 3 1 1 3

[1] "Sum-square Observation vector :  402"


In [15]:
#Means
meanx2 <- mean(obs_x2)
print(paste("Mean Vector : "))
(mean_x2 <- c(rep(meanx2, length(x2t1)), 
              rep(meanx2, length(x2t2)), 
              rep(meanx2, length(x2t3))))

sumsquare_meanx2 <- sum(mean_x2^2)
print(paste("Sum-Square Mean Vector : ", sumsquare_meanx2))

[1] "Mean Vector : "


[1] 5 5 5 5 5 5 5 5 5 5 5 5

[1] "Sum-Square Mean Vector :  300"


In [16]:
#Treatment Effects
x2t1_bar <- mean(x2t1)
x2t2_bar <- mean(x2t2)
x2t3_bar <- mean(x2t3)
treateffect_x2t1 <- x2t1_bar - meanx2
treateffect_x2t2 <- x2t2_bar - meanx2
treateffect_x2t3 <- x2t3_bar - meanx2

treateffect_x2 <- c(rep(treateffect_x2t1, length(x2t1)), 
                     rep(treateffect_x2t2, length(x2t2)), 
                     rep(treateffect_x2t3, length(x2t3)))
print(paste("Treatment Effect Vector : "))
treateffect_x2

sumsquare_treateffectx2 <- sum(treateffect_x2^2)
print(paste("Sum-square Treatment Effect Vector : ", sumsquare_treateffectx2))

[1] "Treatment Effect Vector : "


[1]  3  3  3  3  3 -1 -1 -1 -3 -3 -3 -3

[1] "Sum-square Treatment Effect Vector :  84"


In [17]:
#Residuals
x2bar <- c(rep(x2t1_bar, length(x2t1)), 
                rep(x2t2_bar, length(x2t2)), 
                rep(x2t3_bar, length(x2t3)))

residual_x2 <- obs_x2 - x2bar
print(paste("Residual Effect Vector : "))
residual_x2

sumsquare_residx2 <- sum(residual_x2^2)
print(paste("Sum-square Residual Effect Vector : ", sumsquare_residx2))

[1] "Residual Effect Vector : "


[1] -1  1 -2  1  1 -1  2 -1  1 -1 -1  1

[1] "Sum-square Residual Effect Vector :  18"


### Cross products contributions

In [18]:
crossprodsumsquare_obs <- crossprod(obs_x1, obs_x2)
print(paste("Cross Products of Observations Sum-squares :", crossprodsumsquare_obs))

[1] "Cross Products of Observations Sum-squares : 275"


In [19]:
crossprodsumsquare_mean <- crossprod(mean_x1, mean_x2)
print(paste("Cross Products of Means Sum-squares :", crossprodsumsquare_mean))

[1] "Cross Products of Means Sum-squares : 240"


In [20]:
crossprodsumsquare_corr <- crossprodsumsquare_obs - crossprodsumsquare_mean
print(paste("Total Corrected :", crossprodsumsquare_corr))

[1] "Total Corrected : 35"


In [21]:
crossprodsumsquare_treateffect <- crossprod(treateffect_x1, treateffect_x2)
print(paste("Cross Product of Treatment Sum-Squares :", crossprodsumsquare_treateffect))

[1] "Cross Product of Treatment Sum-Squares : 48"


In [22]:
crossprodsumsquare_residual <- crossprod(residual_x1, residual_x2)
print(paste("Cross Product Residual Sum-Squares :", crossprodsumsquare_residual))

[1] "Cross Product Residual Sum-Squares : -13"


## b). Using the information in Part a, construct the one-way MANOVA table.

In [23]:
SSM_treatment <- matrix(c(sumsquare_treateffectx1, crossprodsumsquare_treateffect,
                          crossprodsumsquare_treateffect, sumsquare_treateffectx2),
                        byrow = T, ncol = 2)
print("Sum-Squares Matrix of Treatment :")
SSM_treatment

df_treatment <- length(obs_x1) - 3
print(paste("Degree of Freedom of Treatment :", df_treatment))

[1] "Sum-Squares Matrix of Treatment :"


36,48
48,84


[1] "Degree of Freedom of Treatment : 9"


In [24]:
SSM_residual <- matrix(c(sumsquare_residx1, crossprodsumsquare_residual,
                         crossprodsumsquare_residual, sumsquare_residx2),
                       byrow = TRUE, ncol = 2)
print("Sum-Squares Matrix of Residual :")
SSM_residual

df_residual <- 3 - 1
print(paste("Degree of Freedom of Residual :", df_residual))

[1] "Sum-Squares Matrix of Residual :"


18,-13
-13,18


[1] "Degree of Freedom of Residual : 2"


In [25]:
SSM_TotalCorrected <- SSM_treatment + SSM_residual
print("Sum-Squares Matrix of Total Corrected :")
SSM_TotalCorrected

df_totalcorrected <- df_treatment + df_residual
print(paste("Degree of Freedom of Total Correction :", df_totalcorrected))

[1] "Sum-Squares Matrix of Total Corrected :"


54,35
35,102


[1] "Degree of Freedom of Total Correction : 11"


## c) Evaluate Wilks’ lambda, A*, and use Table 6.3 to test for treatment effects. Set a = .01. Repeat the test using the chi-square approximation with Bartlett's correction. [See (6-39).] Compare the conclusions.

### (1) Calculating Wilks’ lambda A* with the F-table value

In [26]:
a_star <- det(SSM_residual) / det(SSM_TotalCorrected)
a_star

[1] 0.03618959

In [27]:
g <- 3
p <- 2
n <- length(obs_x1)
alpha01 <- 0.01

Wilk_dist <- ((1 - sqrt(a_star))/sqrt(a_star)) * ((n - g - 1)/(g - 1))
print(paste("Wilks' Lambda A* :", Wilk_dist))

Ftable <- qf(1 - alpha01, df1 = 2 * (g - 1), df2 = 2 * (n - g - 1))
print(paste("Ftable Value : ", Ftable))

[1] "Wilks' Lambda A* : 17.0265577076291"
[1] "Ftable Value :  4.77257799972321"


### (2) Calculating Chi-square approximation test using Bartlett’ correction

In [28]:
BartlettCorr <- - (n - 1 - ((p + g) / 2)) * log(a_star)
print(paste("Bartlett's Correction :", BartlettCorr))

Chitable <- qchisq(1 - alpha01, p * (g - 1))
print(paste("Chi-square Table Value:", Chitable))

[1] "Bartlett's Correction : 28.211362815917"
[1] "Chi-square Table Value: 13.2767041359876"


# 6.19

### Dataset

In [29]:
Diesel<-read_excel("Diesel_Pop.xlsx")
Diesel

fuel_diesel,repair_diesel,capital_diesel
<dbl>,<dbl>,<dbl>
8.50,12.26,9.11
7.42,5.13,17.15
10.28,3.32,11.23
10.16,14.72,5.99
12.79,4.17,29.28
9.60,12.72,11.00
6.47,8.89,19.00
11.35,9.95,14.53
9.15,2.94,13.68


In [30]:
Gas<-read_excel("Gasoline_Pop.xlsx")
Gas

fuel_gas,repair_gas,capital_gas
<dbl>,<dbl>,<dbl>
16.44,12.43,11.23
7.19,2.70,3.92
9.92,1.35,9.75
4.24,5.78,7.78
11.20,5.05,10.67
14.25,5.78,9.88
13.50,10.98,10.60
13.32,14.27,9.45
29.11,15.09,3.28


## (a) Test for differences in the mean cost vectors. Set a = .01. 

In [31]:
n1_19 <- nrow(Diesel)
n1_19

xbar1_19 <- matrix(colMeans(Diesel), ncol = 1)
xbar1_19

S1_19 <- cov(Diesel)
S1_19

[1] 23

10.10565
10.76217
18.16783


,fuel_diesel,repair_diesel,capital_diesel
fuel_diesel,4.3623166,0.7598872,2.362099
repair_diesel,0.7598872,25.8512360,7.685732
capital_diesel,2.3620992,7.6857322,46.654400


In [32]:
n2_19 <- nrow(Gas)
n2_19

xbar2_19 <- matrix(colMeans(Gas), ncol = 1)
xbar2_19

S2_19 <- cov(Gas)
S2_19

[1] 36

12.218611
8.112500
9.590278


,fuel_gas,repair_gas,capital_gas
fuel_gas,23.013361,12.366395,2.906609
repair_gas,12.366395,17.544111,4.773082
capital_gas,2.906609,4.773082,13.963334


In [33]:
mu_19 <- c(0, 0, 0)
T.test <- function(xb1, xb2, S.1, S.2, n1, n2, mu){
  p <- nrow(xb1)
  df1 = p
  df2 = n1 + n2 - p - 1
  alpha <- 0.01
  
  S_pooled <- ((n1 - 1)/(n1 + n2 -2) * S.1) + ((n2 - 1)/(n1 + n2 -2) * S.2)
  
  T_square <- t((xb1 - xb2) - mu) %*% solve(((1/n1) + (1/n2)) * S_pooled) %*% ((xb1 - xb2) - mu)
    
  Fstat <- T_square / (p %*% (n1 + n2 - 2)/(n1 + n2 - p - 1))
  
  Ftable <- qf(1 - alpha, df1 = p, df2 = n1 + n2 - p - 1) %*% (n1 + n2 - 2) %*% p / (n1 + n2 - p - 1)
  
  p_value <- 1 - pf(Fstat, df1 = p, df2 = n1 + n2 - p - 1)
  
  data.frame(T_square = as.numeric(T_square), 
             Fstat = as.numeric(Fstat),
             df1 = p, 
             df2 = n1 + n2 - p - 1, 
             Ftable = as.numeric(Ftable),
             pvalue = as.numeric(p_value),
             row.names="")
}

In [34]:
T.test(xbar1_19, xbar2_19, S1_19, S2_19, n1_19, n2_19, mu_19)

,T_square,Fstat,df1,df2,Ftable,pvalue
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
,50.91279,16.37546,3,55,12.93096,1.000461e-07


## (b) If the hypothesis of equal cost vectors is rejected in Part a, find the linear  combination of mean components most responsible for the rejection. 

In [35]:
Spooled_19 <- ((n1_19 - 1)/(n1_19 + n2_19 -2) * S1_19) + ((n2_19 - 1)/(n1_19 + n2_19 -2) * S2_19)
Spooled_19

,fuel_diesel,repair_diesel,capital_diesel
fuel_diesel,15.814712,7.886690,2.696447
repair_diesel,7.886690,20.750370,5.897263
capital_diesel,2.696447,5.897263,26.580938


In [36]:
linearcomb_19 <- solve(Spooled_19)%*%(xbar1_19-xbar2_19)
linearcomb_19

fuel_diesel,-0.2547452
repair_diesel,0.1339036
capital_diesel,0.3188296


## (c) Construct 99% simultaneous confidence intervals for the pairs of mean components. Which costs, if any, appear to be quite different?

In [37]:
temp_19 <- NULL
alpha <- 0.01
p <- 3
n1 <- n1_19
n2 <- n2_19
Ftable <-  qf((1 - alpha), df1 = p, df2 = n1 + n2 - p - 1) %*% (n1 + n2 - 2) %*% p / (n1 + n2 - p - 1)

for (k in 1:3) {
    temp_19 <- c(temp_19,(xbar1_19[k]-xbar2_19[k])-sqrt(Ftable)*sqrt(((1/n1_19)+(1/n2_19))*Spooled_19[k,k]),
          (xbar1_19[k]-xbar2_19[k])+sqrt(Ftable)*sqrt(((1/n1_19)+(1/n2_19))*Spooled_19[k,k]))
}

rbind(temp_19)

temp_19,-5.930264,1.704346,-1.72292,7.022268,3.628618,13.52648


Berdasarkan selang keperccayaan di atas, dapat disimpulkan bahwa variabel capital_gas adalah satu-satunya variabel yang memiliki perbedaan yang signifikan antara truk diesel dan bensin. (Selang kepercayaan tidak mengandung nilai 0)

## (d) Comment on the validity of the assumptions used in your analysis. Note in particular that observations 9 and 21 for gasoline trucks have been identified as multivariate outliers. (See Exercise 5.22 and [2] .) Repeat Part a with these observations deleted. Comment on the results.

### Deleting observations 9 and 21 as the multivariate outliers

In [38]:
newGas <- Gas[-c(9,21), ]
newGas

fuel_gas,repair_gas,capital_gas
<dbl>,<dbl>,<dbl>
16.44,12.43,11.23
7.19,2.70,3.92
9.92,1.35,9.75
4.24,5.78,7.78
11.20,5.05,10.67
14.25,5.78,9.88
13.50,10.98,10.60
13.32,14.27,9.45
12.68,7.61,10.23


### Redo part a.

In [39]:
newn2_19 <- nrow(newGas)
newn2_19

newxbar2_19 <- matrix(colMeans(newGas), ncol = 1)
newxbar2_19

newS2_19 <- cov(newGas)
newS2_19

[1] 34

11.311765
7.632941
9.561176


,fuel_gas,repair_gas,capital_gas
fuel_gas,9.025021,5.155749,3.201671
repair_gas,5.155749,14.258694,4.318945
capital_gas,3.201671,4.318945,11.987344


In [40]:
T.test(xbar1_19, newxbar2_19, S1_19, newS2_19, n1_19, newn2_19, mu_19)

,T_square,Fstat,df1,df2,Ftable,pvalue
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
,52.3836,16.82625,3,53,12.99521,8.357985e-08


# 6.21

### bond companies

In [41]:
n1_21 = 20
print(paste("n1 :", n1_21))

xbar1_21 <- matrix(c(2.287, 12.600, 0.347, 14.830), nrow = 1)
print(paste("xbar1' :"))
xbar1_21

S1_21 <- matrix(c(0.459, 0.254, -0.026, -0.244, 0.254, 27.465, -0.589, -0.267, -0.026, -0.589, 0.030, 0.102, -0.244, -0.267, 0.102, 6.854), nrow = 4, byrow = TRUE)
print(paste("S1 :"))
S1_21

[1] "n1 : 20"
[1] "xbar1' :"


2.287,12.6,0.347,14.83


[1] "S1 :"


0.459,0.254,-0.026,-0.244
0.254,27.465,-0.589,-0.267
-0.026,-0.589,0.030,0.102
-0.244,-0.267,0.102,6.854


### Baa bond companies

In [42]:
n2_21 = 20
print(paste("n2 :", n2_21))

xbar2_21 <- matrix(c(2.404, 7.155, 0.524, 12.840), nrow = 1)
print(paste("xbar2' :"))
xbar2_21

S2_21 <- matrix(c(0.944, -0.089, 0.002, -0.719, -0.089, 16.432, -0.400, 19.044, 0.002, -0.400, 0.024, -0.094, -0.719, 19.044, -0.094, 61.854), nrow = 4, byrow = TRUE)
print(paste("S2 :"))
S2_21

[1] "n2 : 20"
[1] "xbar2' :"


2.404,7.155,0.524,12.84


[1] "S2 :"


0.944,-0.089,0.002,-0.719
-0.089,16.432,-0.400,19.044
0.002,-0.400,0.024,-0.094
-0.719,19.044,-0.094,61.854


### Spooled

In [43]:
Spooled_21 <- matrix(c(0.701, 0.083, -0.012, -0.481, 0.083, 21.949, -0.494, 9.388, -0.012, -0.494, 0.027, 0.004, -0.481, 9.388, 0.004, 34.354), nrow = 4, byrow = TRUE)
Spooled_21

0.701,0.083,-0.012,-0.481
0.083,21.949,-0.494,9.388
-0.012,-0.494,0.027,0.004
-0.481,9.388,0.004,34.354


## (a) Does pooling appear reasonable here? Comment on the pooling procedure in this case. 

## (b) Are the financial characteristics of firms with Aa bonds different from those with Baa bonds? Using the pooled covariance matrix, test for the equality of mean vectors. Set a == .05. 

In [44]:
mu_21 <- c(0, 0, 0, 0)
T.test <- function(xb1, xb2, S_pooled, n1, n2, mu){
  p <- nrow(xb1)
  df1 = p
  df2 = n1 + n2 - p - 1
  alpha <- 0.05
  
  T_square <- t((xb1 - xb2) - mu) %*% solve(((1/n1) + (1/n2)) * S_pooled) %*% ((xb1 - xb2) - mu)
    
  Fstat <- T_square / (p %*% (n1 + n2 - 2)/(n1 + n2 - p - 1))
  
  Ftable <- qf(1 - alpha, df1 = p, df2 = n1 + n2 - p - 1) %*% (n1 + n2 - 2) %*% p / (n1 + n2 - p - 1)
  
  p_value <- 1 - pf(Fstat, df1 = p, df2 = n1 + n2 - p - 1)
  
  data.frame(T_square = as.numeric(T_square), 
             Fstat = as.numeric(Fstat),
             df1 = p, 
             df2 = n1 + n2 - p - 1, 
             Ftable = as.numeric(Ftable),
             pvalue = as.numeric(p_value),
             row.names="")
}

In [45]:
T.test(t(xbar1_21), t(xbar2_21), Spooled_21, n1_21, n2_21, mu_21) 

,T_square,Fstat,df1,df2,Ftable,pvalue
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
,15.83653,3.646571,4,35,11.47151,0.01383831


## (c) Calculate the linear combinations of mean components most responsible for rejecting H0 : ILl - IL2 == 0 in Part b. 

In [46]:
linearcomb_21 <- solve(Spooled_21) %*% (t(xbar1_21) - t(xbar2_21)) 
linearcomb_21

-0.24210856
0.16013330
-3.73497395
0.01121134


## (d) Bond rating companies are interested in a company's ability to satisfy its outstanding debt obligations as they mature. Does it appear as if one or more of the foregoing financial ratios might be useful in helping to classify a bond as "high" or "medium" quality? Explain. 

# 6.23


## Inputing Dataset

In [47]:
iris

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa


## (2)	Testing for the MANOVA One-Way Test with Ho : Τ1 = Τ2 = Τ3 = 0 (no treatment effect)

In [48]:
X <- as.matrix(iris[,c("Sepal.Width","Petal.Width")])
species <- as.factor(iris$Species)
model <- lm(X ~ species)
widthManova <- manova(model)
widthManova

Call:
   manova(model)

Terms:
                 species Residuals
Sepal.Width     11.34493  16.96200
Petal.Width     80.41333   6.15660
Deg. of Freedom        2       147

Residual standard errors: 0.3396877 0.20465
Estimated effects may be unbalanced

In [49]:
sumWidth <- summary(widthManova, test="Wilks")
sumWidth

           Df    Wilks approx F num Df den Df    Pr(>F)    
species     2 0.038316   299.94      4    292 < 2.2e-16 ***
Residuals 147                                              
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

In [50]:
sumWidth$SS

,Sepal.Width,Petal.Width
Sepal.Width,11.34493,-22.93267
Petal.Width,-22.93267,80.41333
,Sepal.Width,Petal.Width
Sepal.Width,16.9620,4.8084
Petal.Width,4.8084,6.1566
